# 19 - Classificação de imagens com Bag of Visual Words

Este roteiro complementa a aula 17 e mostra alguns exemplos iniciais de classificação de imagens usando o modelo bag of words.

# Parte 0 - revisão

Crie funções auxiliares usando o conteúdo da *Atividade 2*:

1. `def cria_vocabulario(path_images)`: recebe uma lista de imagens de treinamento e devolve um objeto `sklearn.KMeans` contendo os centróides de cada palavra no vocabulário.
2. `def representa(vocab, imagem)`: cria a representação em histograma de uma imagem usando os centróides do objeto `vocab` (criado pela função acima)

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import namedtuple
import os, sys
from random import shuffle
from sklearn.cluster import KMeans


def computa_descritores(img):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(img, None)    
    return des

def le_descritores_imagens(pastas, max_items = 5):
    list_Names  = []
    list_Matrix = []
    list_Imgs   = []
    Tuple = namedtuple('Tupla', 'listNames, matrix ')
    
    for pasta in pastas:
        dir_Name = "../aula17/101_ObjectCategories/" + pasta + '/'
        img_Name = os.listdir(dir_Name)
        #shuffle(img_Name)
            
        for i in range(max_items):
            if(img_Name[i][0]=="."):
                img_Name[i]=img_Name[i][2:]
            name = dir_Name + img_Name[i]
            list_Imgs.append(cv2.imread(name))
            list_Names.append(name)
            
    for img in list_Imgs:
        list_Matrix.append(computa_descritores(img))
    
    tup = Tuple(list_Names, np.concatenate(list_Matrix)) 
    return tup

def cria_vocabulario(descritores, sz = 300):                

    Tuple = namedtuple('Tupla', 'matrix kmeans')
    kmeans = KMeans(n_clusters = sz, random_state=0).fit(descritores)
    tup = Tuple(kmeans.cluster_centers_ , kmeans)

    return tup

def representa_histograma(img, vocab):
    img = cv2.imread(img)
    desc = computa_descritores(img)
    
    dist = vocab.kmeans.predict(desc)
    
    list_freq = [0] * 300
    
    for i in dist:
        list_freq[i] += 1
   
    return list_freq

# Parte 1 - Classificação Binária

Nesta parte iremos usar as imagens da base *Caltech 101* usada na *Atividade 2*. Em especial, iremos usar as 50 primeiras imagens das seguintes pastas: `[dalmatian, Faces_easy]`. Ou seja, diferenciaremos cachorros de rostos.

**Exercício**: crie uma lista dos caminhos das primeiras 50 imagens das duas pastas.

In [8]:
folder = ['dalmatian', 'Faces_easy']
descriptor = le_descritores_imagens(folder)

**Exercício**: use suas funções da parte 0 para criar um vocabulário e representar cada imagem da lista como um histograma. Salve seu resultado em uma matriz **X** tal que a linha *i* da matriz seja o histograma da imagem *i* na lista acima. 

Como já visto em ciência dos dados, é muitas vezes importante transformar nossos dados para que eles tenham média 0 e variância 1. 

**Exercício**: Faça esse transformação em **X** e salve na variável **X_std**. Não se esqueça de guardar, explicitamente, os valores de média e variância encontrados. 

*Dica*: o scikit-learn possui a classe [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) que já faz exatamente isto. Você pode usá-la neste exercício se quiser. 

Agora que temos uma representação para cada imagem, precisamos criar uma lista contendo os rótulos corretos de cada imagem.

**Exercício**: crie um vetor **y** tal que o índice *i* seja -1 se a imagem *i* na lista for um rosto e *1* se a imagem *i* for um cachorro.

Vamos agora treinar um classificador para diferenciar rostos de cachorros. A entrada do nosso classificador é uma matriz **X_std** contendo os histogramas das imagens e a saída esperada será o vetor de rótulos **y**. Se possível, gostaríamos de inferir o rótulo a partir do histograma. 

Neste exercício usaremos novamente a biblioteca *Scikit Learn*, desta vez com a classe `sklearn.svm.SVC`. Veja [neste link](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) a documentação. Estamos particularmente interessados nos métodos `fit` (treina o classificador a partir dos dados acima) e `predict` (classifica uma imagem dado seu histograma).

**Exercício**: crie um objeto do tipo `SVC` e treine-o usando as informações obtidas anteriormente. 

**Exercício**: use o método `predict` para predizer a classe da primeira e da última imagem e interprete os resultados. Não se esqueça de usar os valores armazenados em **X_std**.

**Exercício**: o que faz o método `score` de `SVC`? Use-o e comente seus resultados.

# Parte 2 - Avaliação de desempenho

Vamos agora avaliar a qualidade de nosso classificador de imagens. Para isto iremos usar as 5 pŕoximas imagens das pastas da parte 1. Nossa taxa de acerto será dada pelo número de imagens classificadas corretamente. Note que, para termos uma estimativa boa do acerto precisamos usar imagens totalmente novas. 

**Exercício**: usando o classificador treinado na parte 1, leia as imagens do disco e, para cada imagem, classifique-a em cachorro ou rosto. Salve os resultados em um vetor.

**Importante**: antes de enviar nossos dados para o classificador fizemos a transformação para deixá-los com média 0 e variância 1. Isto deve ser feito com cada histograma antes de usar o `predict` usando os valores de média e variância encontrados anteriormente. 

**Exercício**: baseado nos resultados acima, calcule a taxa de acerto do seu classificador.

**Exercício**: calcule, agora, a taxa de erro das imagens usadas no treinamento do classificador. Seus resultados foram parecidos? 